In [1]:
import pandas as pd
from IPython.display import display
import argparse
import os
from aggregate_entity_results import collect_results, aggregate_results
from utils import generate_explanation_prediction

In [2]:
data_dir = '/scratch/dzhang5/LLM/TWEET-FID/'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/unlabeled-results-autolabel-ner-qa-mv/0.1'
model_name = "refuel-llm"
few_shot_selection = "semantic_similarity"
text_column = "context"
label_symbol = "^^^^"
verified = False
result_dir = '/scratch/dzhang5/LLM/TWEET-FID/unlabeled-results-autolabel-ner-qa-mv/0.1'
unlabeled_dir = '/scratch/dzhang5/LLM/TWEET-FID/second-round/'

In [3]:
def process_multiple_files():
    ori_df = pd.read_csv(data_path)
    predictions_list = []
    for label_type in ['Food', 'Location', 'Symptom', 'Keyword']:
        if verified:
            output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_final_' + os.path.split(data_path)[-1]
        else:
            output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + label_type + '_' + os.path.split(data_path)[-1]
        output_path = os.path.join(output_dir, output_name)
        predictions = pd.read_csv(output_path)
        predictions_list.append(predictions)
    merged_predictions = predictions_list[0]
    for i in range(1, len(predictions_list)):
        next_prediction = predictions_list[i]
        assert (merged_predictions[text_column] == next_prediction[text_column]).all()
        cols_to_merge = next_prediction.columns.difference(merged_predictions.columns)
        merged_predictions = merged_predictions.join(next_prediction[cols_to_merge], validate='1:1')
    assert (ori_df['context'] == merged_predictions['context']).all()
    merged_predictions['All_answer_successfully_labeled'] = (merged_predictions['Food_answer_successfully_labeled'] & merged_predictions['Location_answer_successfully_labeled'] & merged_predictions['Symptom_answer_successfully_labeled'] & merged_predictions['Keyword_answer_successfully_labeled'])
    for label_type in ['Food', 'Location', 'Symptom', 'Keyword', 'All']:
        display(merged_predictions[f'{label_type}_answer_successfully_labeled'].value_counts())
    unlabeled_dict = {}
    for label_type in ['Food', 'Location', 'Symptom', 'Keyword']:
        unlabeled_dict[label_type] = ori_df.loc[~merged_predictions[f'{label_type}_answer_successfully_labeled']].copy()
        unlabeled_dict[label_type].reset_index(inplace=True)
        unlabeled_dict[label_type].rename(columns={'index':'ori_index'}, inplace=True)
    labeled_df = ori_df.loc[merged_predictions['All_answer_successfully_labeled']].copy()
    labeled_df.reset_index(inplace=True)
    labeled_df.rename(columns={'index':'ori_index'},inplace=True)
    keep_predictions = merged_predictions.loc[merged_predictions['All_answer_successfully_labeled']].copy()
    words_list = []
    for label_type in ['Food', 'Location', 'Symptom', 'Keyword']:
        candidate_words = collect_results(keep_predictions, text_column, label_type, label_symbol)
        words_list.append(candidate_words)
    labeled_df['CategorizedLabels_prediction'] = aggregate_results(words_list)
    labeled_df['sentence_explanation_prediction'] = labeled_df['CategorizedLabels_prediction'].apply(lambda x: generate_explanation_prediction(x, 'N/A'))
    agg_output_suffix_name = os.path.split(data_path)[-1].replace('unlabeled', 'unlabeled-first')
    unlabeled_output_suffix_name = os.path.split(data_path)[-1].replace('unlabeled', 'unlabeled-second')
    if verified:
        agg_output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_aggregated_final_' + agg_output_suffix_name
    else:
        agg_output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_aggregated_' + agg_output_suffix_name
    agg_output_path = os.path.join(result_dir, agg_output_name)
    print(agg_output_path)
    labeled_df.to_csv(agg_output_path, index=False)
    for label_type in ['Food', 'Location', 'Symptom', 'Keyword']:
        unlabeled_output_name = f'{label_type}-{unlabeled_output_suffix_name}'
        unlabeled_output_path = os.path.join(unlabeled_dir, unlabeled_output_name)
        print(unlabeled_output_path)
        unlabeled_dict[label_type].to_csv(unlabeled_output_path, index=False)
    return len(labeled_df)

In [4]:
count_labeled = 0
missing_file = 0
for ix in range(1,21):
    data_name = f'unlabeled-{ix}.csv'
    data_path = os.path.join(data_dir, data_name)
    try:
        count_labeled += process_multiple_files()
    except FileNotFoundError:
        print(f'{data_name} has missing files')
        missing_file +=1
        continue
print("Total Labeled:", count_labeled, 'missing files:', missing_file)

Food_answer_successfully_labeled
True     1364
False    1136
Name: count, dtype: int64

Location_answer_successfully_labeled
True     1283
False    1217
Name: count, dtype: int64

Symptom_answer_successfully_labeled
True     1388
False    1112
Name: count, dtype: int64

Keyword_answer_successfully_labeled
True     1367
False    1133
Name: count, dtype: int64

All_answer_successfully_labeled
False    2191
True      309
Name: count, dtype: int64

...

Location_answer_successfully_labeled
True     1706
False     794
Name: count, dtype: int64

Symptom_answer_successfully_labeled
True     1847
False     653
Name: count, dtype: int64

Keyword_answer_successfully_labeled
True     1841
False     659
Name: count, dtype: int64

All_answer_successfully_labeled
False    2006
True      494
Name: count, dtype: int64

aggregate entities results ...
aggregate entities results ...
aggregate entities results ...
aggregate entities results ...
/scratch/dzhang5/LLM/TWEET-FID/unlabeled-results-autolabel-ner-qa-mv/0.1/refuel-llm_semantic_similarity_aggregated_unlabeled-first-20.csv
/scratch/dzhang5/LLM/TWEET-FID/second-round/Food-unlabeled-second-20.csv
/scratch/dzhang5/LLM/TWEET-FID/second-round/Location-unlabeled-second-20.csv
/scratch/dzhang5/LLM/TWEET-FID/second-round/Symptom-unlabeled-second-20.csv
/scratch/dzhang5/LLM/TWEET-FID/second-round/Keyword-unlabeled-second-20.csv
Total Labeled: 8955 missing files: 0
